In [51]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [52]:
import os
import json
import time
import pandas as pd
import requests

from langchain import hub
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import logging
from openai import OpenAI
import traceback


In [53]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
UPSTAGE_API_KEY = os.environ.get('UPSTAGE_API_KEY')
LANGCHAIN_API_KEY = os.environ.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = 'EXP04_GC' # 프로젝트명 수정
LANGCHAIN_PROJECT = os.environ.get('LANGCHAIN_PROJECT')

print(f'LangSmith Project: {LANGCHAIN_PROJECT}')

LangSmith Project: EXP04_GC


In [54]:
load_dotenv()

True

In [55]:
# 데이터 구성
file_path = '../data/documents.jsonl'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True,
)
temp = loader.load()

seq_num = 1
documents = []
for tmp in temp:
    data = json.loads(tmp.page_content)
    doc = Document(page_content=data['content'], metadata={
        'docid': data['docid'],
        'src': data['src'],
        'source': '/data/ephemeral/home/upstage-ai-final-ir2/upstage-ai-final-ir2/HM/data/documents.jsonl',
        'seq_num': seq_num,
    })
    documents.append(doc)
    seq_num += 1


In [56]:
# Splitter
splitter = CharacterTextSplitter(
    separator='',
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
split_documents = splitter.split_documents(documents)

In [57]:
# Embedding
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY, 
    model="solar-embedding-1-large"
)

In [58]:
# 벡터 저장소 생성
# pip install faiss-cpu
folder_path = f'./faiss_{LANGCHAIN_PROJECT}'
if not os.path.exists(folder_path):
    print('Vector Store 생성 중')
    vectorstore = FAISS.from_documents(
        documents=split_documents,
        embedding=embeddings,
    )
    vectorstore.save_local(folder_path=folder_path)
    print('Vector Store 생성 및 로컬 저장 완료')
else:
    vectorstore = FAISS.load_local(
        folder_path=folder_path, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True
    )
    print('Vector Store 로컬에서 불러옴')


Vector Store 로컬에서 불러옴


In [59]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
prompt = hub.pull("rlm/rag-prompt")

In [60]:
# LLM과 검색엔진을 활용한 RAG 구현 (기존 코드와 동일)
retriever = vectorstore.as_retriever(k=5)
chat = ChatUpstage(model='solar-1-mini-chat', temperature=0)

In [61]:
client = OpenAI()
llm_model = "gpt-3.5-turbo"

In [131]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
search_mode = ""

persona_qa = """
## Role: 사회/과학 지식 전문가

## Instructions
사용자의 질문을 보고 다음 두가지 행동 중 하나를 취한다. 
1. 사용자가 대화를 통해 사회/과학 지식에 관한 주제로 질문하면 search api를 호출한다. 
2. 사회/과학 지식과 관련되지 않은 나머지 대화 메시지에는 대답을 하지 않고 검색도 하지 않는다. 
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 사회/과학 지식 전문가

## Instruction
사용자의 질문을 보고 다음 두가지 행동 중 하나를 취한다. 

1. 사용자가 대화를 통해 사회/과학 지식에 관한 주제로 질문하면 search_mode="on"
   - 질문이 물리, 화학, 생물, 지구과학, 천문학, 컴퓨터공학, 소프트웨어, 국제정치 등 지식을 묻는 것이라면 search api를 호출할 수 있어야 한다. 
   - 특히, "알려줘", "설명해줘"로 끝나는 query는 search api를 호출한다. 
   - 아래는 search api를 호출하는 질문의 예시이다. 
   - 예시:
    "메탄올의 화학식은 무엇인가요?", 
    "달의 공전 주기는 얼마인가요?", 
    "파이썬에서 여러 값들의 평균을 구하는 방법은?",
    "이란의 무기 리베이트가 미국 정치에 미치는 영향은?",
    "각 나라별 공교육 현황은?", 
    "대중교통은 그 가치가 얼마나 될까?", 
    "각 나라의 문해율에 대해 알려줘", 
    "화성에서 살 수 있을까?", 
    "Edison"이 누구야?" 


2. 사용자의 대화 메시지가 사회/과학 지식과 관련되지 않은 경우 search_mode="off"
   - 개인적인 감정이나 간단한 인사 등은 search api를 호출하지 않는다. 
   - 아래는 대답을 하지 않는 질문의 예시이다.
   - 예시: 
    "오늘은 기분이 아주 나빠", 
    "너는 모든 것을 알고 있구나", 
    "대답을 잘해줘서 너무 고마워!", 
    "요새 기분이 들쭉날쭉해",
    "안녕 너 이름이 뭐야",
    "너는 지금 무엇을 하고 있어?", 
    "우리 재미있는 이야기 하자." 
    """
# if search_mode == "on":
#     # search api를 호출하여 검색 
# else:
#     # search api를 호출하지 않고 대답하지 않음 


# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Search relevant documents based on user message history",
            "parameters": {
                "properties": {
                    "message_history": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "role": {"type": "string"},
                                "content": {"type": "string"}
                            },
                            "required": ["role", "content"]
                        },
                        "description": "The entire history of user messages to form a comprehensive search query."
                    }
                },
                "required": ["message_history"],
                "type": "object"
            }
        }
    },
]


In [132]:
# Groundedness Check 함수 추가
def check_groundedness(context, response):
    url = "https://api.upstage.ai/v1/solar/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ.get('UPSTAGE_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "solar-1-mini-groundedness-check",
        "messages": [
            {"role": "user", "content": context},
            {"role": "assistant", "content": response}
        ],
        "temperature": 0
    }
    
    try:
        api_response = requests.post(url, headers=headers, json=data)
        api_response.raise_for_status()  # Raises an HTTPError for bad responses
        result = api_response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {e}")
        return "Error in groundedness check"

In [133]:
def format_docs(docs):
    global references
    references = docs
    return '\n\n'.join(doc.page_content for doc in docs)

In [134]:
def answer_question(messages):
    global search_mode, references
    response = {"topk": "", "answer": "", "references": "", "groundedness": ""}

    try:
        # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
        msg = [{"role": "system", "content": persona_function_calling}] + messages
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            temperature=0,
            seed=1,
            timeout=10
        )

        # 검색이 필요한 경우 RAG 체인 활용
        if search_mode == "on":
            rag_chain = (
                {'context': retriever | format_docs, 'question': RunnablePassthrough()}
                | prompt
                | chat
                | StrOutputParser()
            )
            history = '\n'.join([f"{message['role']}: {message['content']}" for message in messages]) + '\n'
            response["answer"] = rag_chain.invoke(history)
            ref_content = [reference.page_content for reference in references]
            topk = [reference.metadata['docid'] for reference in references]
            
        # 검색이 필요하지 않은 경우 바로 답변 생성
        else:
            response["answer"] = result.choices[0].message.content
            ref_content = []
            topk = []

    except Exception as e:
        traceback.print_exc()
        return response

    # Groundedness Check 수행
    context = '\n'.join(ref_content)
    groundedness = check_groundedness(context, response["answer"])

    response["topk"] = topk
    response["references"] = ref_content
    response["groundedness"] = groundedness

    return response

In [135]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as eval_lines, open(output_filename, 'w') as output_lines:
        idx = 0
        for eval_line in eval_lines:
            j = json.loads(eval_line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}')
            print(f'Groundedness: {response["groundedness"]}\n')

            output = {
                "eval_id": j["eval_id"], 
                "question": j["msg"],  # 질문 전체 저장
                # "standalone_query": response["standalone_query"],
                "topk": response["topk"], 
                "answer": response["answer"], 
                "references": response["references"],
                "groundedness": response["groundedness"]
            }
            output_lines.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [136]:
# 평가 실행
eval_rag('../data/eval.jsonl', '../output/EXP04_GC.csv')

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: "나무의 분류에 대해 조사해 보기 위한 방법은 다양합니다. 일반적으로 나무의 분류는 생물학적 특성에 따라 이루어지며, 주요한 방법으로는 다음과 같은 것들이 있습니다.

1. 잎의 형태와 배치: 나무의 잎의 형태와 배치를 관찰하여 분류할 수 있습니다. 예를 들어, 잎의 형태가 날카로운가 둥근가, 잎이 대칭적인가 비대칭적인가 등을 고려합니다.

2. 꽃과 열매: 꽃과 열매의 형태, 색상, 크기 등을 관찰하여 나무를 분류할 수 있습니다. 꽃과 열매의 특징은 해당 나무의 종을 식별하는 데 도움이 됩니다.

3. 줄기와 가지의 형태: 나무의 줄기와 가지의 형태, 질감, 색상 등을 살펴보고 비교하여 분류할 수 있습니다. 일부 나무는 특이한 줄기나 가지의 형태를 가지고 있어 식별이 용이합니다.

4. 나무의 크기와 생장양상: 나무의 크기, 생장양상, 성장 속도 등을 고려하여 분류할 수 있습니다. 일부 나무는 특정한 크기나 생장양상을 가지고 있어 특이한 특징으로 식별됩니다.

이외에도 DNA 분석을 통한 분류, 전문가의 도움을 받는 방법 등이 있을 수 있습니다. 나무의 분류를 위해서는 관련 서적이나 온라인 자료를 참고하거나 전문가의 조언을 구하는 것이 도움이 될 수 있습니다."
Groundedness: grounded

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
API 요청 오류: 400 Client Error: Bad Request for url: https://api.upstage.ai/v1/solar/chat/completions
Answer: None
Groundedness: Error in groundedness check

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리